<a href="https://colab.research.google.com/github/ggeorgekkariuki/Tensorflow-Plant-Classifier/blob/main/TF_Plant_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 2.x  # this line is not required unless you are in a notebook

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x  # this line is not required unless you are in a notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.


In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals


import tensorflow as tf

import pandas as pd

### The Objective

The objective of the project is to classify plants according to certain characteristics of the leaf.

#### The Data Set

This specific dataset seperates flowers into 3 different classes of species.

 - Setosa
 - Versicolor
 - Virginica
 
The information about each flower is the following.

 - sepal length
 - sepal width
 - petal length
 - petal width

In [5]:
CSV_COLUMN_NAMES = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']
# Lets define some constants to help us later on

In [6]:
train_path = tf.keras.utils.get_file(
    "iris_training.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv")
test_path = tf.keras.utils.get_file(
    "iris_test.csv", "https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv")

train = pd.read_csv(train_path, names=CSV_COLUMN_NAMES, header=0)
test = pd.read_csv(test_path, names=CSV_COLUMN_NAMES, header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

16384/573 [=========================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================================] - 0s 0us/step


In [7]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


Remove the 'SPECIES' column that will be our label.

In [8]:
train_y = train.pop('Species')
test_y = test.pop('Species')

In [9]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [11]:
train_y.shape

(120,)

### Preprocessing

#### The Input Function

In [18]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

#### Feature Columns

In [12]:
my_feature_columns = []

for fc in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=fc))

for i in my_feature_columns:
  print(i)

NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)
NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)


### Build the model

We will use the Deep Neural Network Classifier from tf.estimator.DNNClassifier.

This layer will use 2 hidden layers with 30 and 10 nodes respectively

In [25]:
classifier = tf.estimator.DNNClassifier(
    hidden_units = [30, 10],
    feature_columns=my_feature_columns,
    n_classes=3 # based on the plant classes tht there are.
)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpolnqwrkw', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


### Training

Now it's time to train the model!

In [28]:
classifier.train(
    input_fn= lambda: input_fn(train, train_y, training=True),
    steps=10000
)
# We include a lambda to avoid creating an inner function previously

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpolnqwrkw/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmpolnqwrkw/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.45991808, step = 5000
INFO:tensorflow:global_step/sec: 220.269
INFO:tensorflow:loss = 0.44698358, step = 5100 (0.459 sec)
INFO:tensorflow:global_step/sec: 257.829
INFO:tensorflow:loss = 0.45742542, step = 5200 (0.385 sec)
INFO:tensorflow:global_step/sec: 251.79
INFO:tensorflow:loss = 0.44380224, step = 5300 (0.401 sec)
INFO:tensorflow:global_step/sec: 255.14

### Evaluation

In [29]:
eval_result = classifier.evaluate(
    input_fn= lambda: input_fn(test, test_y, training=False)
)
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-12-24T18:56:23
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpolnqwrkw/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.30127s
INFO:tensorflow:Finished evaluation at 2021-12-24-18:56:23
INFO:tensorflow:Saving dict for global step 15000: accuracy = 0.96666664, average_loss = 0.31797117, global_step = 15000, loss = 0.31797117
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 15000: /tmp/tmpolnqwrkw/model.ckpt-15000

Test set accuracy: 0.967



Observation:

With 5000 steps the score was very low - Meaning only about half are right.

Increasing the steps to 10000 produces a great accuracy score, and a low loss figure, but there could be overfitting in the training.

### Predictions
Now that we have a trained model it's time to use it to make predictions.

In [48]:
def input_fn(features, batch_size=256):
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
predict = {}

print("Type numeric values as prompted")
for i in features:
  valid=True
  while valid:
    val = input(i + ': ')
    if not val.isdigit(): valid=False

  predict[i] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))

for pred_dict in predictions:
  class_id = pred_dict['class_ids'][0] # Picks the first list in the list of lists
  probability = pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({:.1f}%)'.format(
        SPECIES[class_id], 100 * probability))


Type numeric values as prompted
SepalLength: 6.9
SepalWidth: 3.1
PetalLength: 5.4
PetalWidth: 2.1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpolnqwrkw/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Prediction is "Virginica" (69.7%)


expected = ['Setosa', 'Versicolor', 'Virginica']

predict_x = {
-    'SepalLength': [5.1, 5.9, 6.9],
-   'SepalWidth': [3.3, 3.0, 3.1],
-   'PetalLength': [1.7, 4.2, 5.4],
-   'PetalWidth': [0.5, 1.5, 2.1],
}

In [45]:
# for pred_dict in predictions:
#   class_id = pred_dict['class_ids'][0] # Picks the first list in the list of lists
#   probability = pred_dict['probabilities'][class_id]

#   print('Prediction is "{}" ({:.1f}%)'.format(
#         SPECIES[class_id], 100 * probability))

for i in predictions:
  print(i)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpolnqwrkw/model.ckpt-15000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([-0.09987579,  2.562227  ,  1.8665528 ], dtype=float32), 'probabilities': array([0.04450082, 0.6375358 , 0.3179634 ], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}


{
  'logits': array([ 4.694751 ,  2.4289007, -1.8772026], dtype=float32),'probabilities': array([0.90486205, 0.09387203, 0.00126596], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}


In [46]:
for pred_dict in predictions:
  classid = pred_dict['class_ids'][0]
  probs = pred_dict['probabilities'][classid]
  
  print(SPECIES[classid], 'predicted with', probs*100, '% probability')